#Considerações

1. Tensão de cisalhamento
==========================

##$\tau = \frac{FD}{2}$

##$\tau = \frac{8FD}{\pi d^3} + \frac{4F}{\pi d^2}$

##$\tau = K_s\frac{8FD}{\pi d^3}$

$K_s$ é um fator de correção da tensão de cisalhamento.
Usando o fator Bergsträsser, temos que:

##$\tau = K_B\frac{8FD}{\pi d^3}$
Onde,

$K_B = \frac{4C + 2}{4C - 3}$;

$C = \frac{D}{d}$, d = diâmetro do fio tabelado;

$4\leq C \leq 12$.


2. Ponto de operação
======================

O percurso fracionário até o fechamento da mola deve ser contido aos 75%.

##$F_s = (1 + \xi)F_{max}$
Onde
##$\xi  \geq 0.15$
Portanto, a tensão de cisalhamento:
##$\tau_s = K_B\frac{8(1 + \xi)F_{max}D}{\pi d^3}$

3. A cargo do projetista
========================

+ Escolher diâmetro $d$ do fio na tabela
+ O diâmetro médio “D” pode ser obtido a partir de $C$
+ Fator Bergsträsser $K_B$ é obtido a partir de $C$
+ $F_{max}$ ou $F$ são conhecidos
+ **Índice $C$ da mola**
+ **$\xi \geq 0.15$ deve ser seguido**

#Resistência a Compressão

Resistência a tração:

##$S_{ut} = \frac{A}{d^m}$

$S_{ut}$: Resistência à tração

𝐴 : Constante (obtida na Tabela)

𝑚: expoente (obtido na Tabela)

Resistência ao escoamento por cisalhamento:

##$S_{sy} = x * S_{ut}$

$x$: Constante da Tabela Porcentagem Máxima de resistência de Tração

1. Coeficiente de segurança
===========================

##$n_s = \frac{S_{sy}}{\tau_s}$

+ **$n_s$: Coeficiente de segurança deve ser $\geq 1.2$**

#Considerações Adicionais (opcional)

1. Rigidez da mola:

##$k = \frac{F}{y}$
e,

##$N_a = \frac{d^4G}{8D^3k}$

onde,

**$N_a$: Número de espiras ativas, deve respeitar $3 \leq N_a \leq 15$.**

$G$: Módulo de elasticidade transversal

In [46]:
import numpy as np
import tkinter as tk
from tkinter import ttk

############################################# Classe Material
class Material:
    def __init__(self, nome, diametros_fio, expoente_m, const_material, percent_max_Sut):
        """
        Inicializa um novo objeto Material.
        :param id: identificação do material
        :param diametros_fio: Lista de valores de diâmetros de fio em mm.
        :param expoente_m: Valor do expoente m.
        :param A: Valor de A em MPa*mm**m.
        :param percent_max_Sut: Porcentagem máxima de resistência de tração.
        """
        self.id = nome
        self.diametros_fio = diametros_fio
        self.m = expoente_m
        self.A = const_material # Constante material
        self.x = percent_max_Sut

    def __str__(self):
        return (f"Material {self.id} com diâmetros: {self.diametros_fio}, "
                f"m: {self.m}, A: {self.A}, "
                f"Porcentagem Máxima de Resistência de Tração: {self.x * 100}%")
    
def getMaterialById(id, materialList):
    for mat in materialList:
        if mat.id == id:
            return mat
    return None

# Materiais disponíveis
material_A227 = Material(
    nome="A227",
    diametros_fio = [
    0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.4, 1.6, 1.8, 2.0,
    2.2, 2.5, 2.8, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0,
    6.5, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0
    ],
    expoente_m=0.190,
    const_material=1783,
    percent_max_Sut=45/100
)

material_A228 = Material(
    nome="A228",
    diametros_fio = [
    0.1, 0.12, 0.16, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5,
    0.55, 0.6, 0.65, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.4,
    1.6, 1.8, 2.0, 2.2, 2.5, 2.8, 3.0, 3.5, 4.0, 4.5, 
    5.0, 5.5, 6.0, 6.5
    ],
    expoente_m=0.145,
    const_material=2211,
    percent_max_Sut=45/100
)

available_materials = [material_A227, material_A228]

############################################# Classe Mola
class Mola:
    def __init__(self):
        """
        Inicializa um novo objeto Mola.
        :param material: objeto material
        :param d: diâmetro do fio em mm.
        :param D: diâmetro da mola em mm.
        :param C: Índice da mola.
        :param force: força atuante na mola.
        :param n_s: coeficiente de segurança.
        """
         
        self.material = None
        self.d = -1
        self.D = -1
        self.C = -1
        self.force = None
        self.n_s = 0

        self.__xi = 0.15
        self.__lockedSafetyFactor = 1.2

    @property
    def xi(self):
        return self.__xi
    
    @xi.setter
    def xi(self, value):
        if value >= 0.15:
            self.__xi = value

    @property
    def lockedSafetyFactor(self):
        return self.__lockedSafetyFactor
    
    @lockedSafetyFactor.setter
    def lockedSafetyFactor(self, value):
        if value >= 1.2:
            self.__lockedSafetyFactor = value

    def update_spring(self):
        if self.d > 0 and self.C > 0:
            self.D = self.getSpringDiam()
            print(f"Diâmetro da mola (D): {self.D} mm")
            self.perform_calculations()

        elif self.d > 0 and self.force != None:
            self.suggestC()

    def perform_calculations(self):
        if self.D > 0 and self.force != None:

            K_B = self.getBergtrasserFactor()
            shearStress = self.getShearStress(self.force)
            tensileStrength = self.getTensileStrength()
            shearYieldStrength = self.getShearYieldStrength()
            safetyFactor = self.getSafetyFactor()

            print(f"Bergtrasser: {K_B}")
            print(f"Shear Stress: {shearStress} MPa")
            print(f"Tensile Strength: {tensileStrength} MPa")
            print(f"Shear Yield Strength: {shearYieldStrength} MPa")
            print(f"Safety Factor: {safetyFactor}")
            self.suggestC()

    def getSpringDiam(self):
        """
        Calcula e retorna o diâmetro da mola.
        """
        return self.C * self.d

    def getBergtrasserFactor(self):
        """
        Calcula e retorna o fator de correção de Bergstrasser (K_B).
        """
        return (4 * self.C + 2) / (4 * self.C - 3)

    def getShearStress(self, Forca):
        """
        Calcula e retorna a tensão de cisalhamento na mola.
        :param Forca: A força aplicada na mola.
        T_s = K8(1 + xi)FD/(pi d^3)
        """
        K = self.getBergtrasserFactor()
        return K * 8 * (1 + self.xi) *Forca * self.D / (np.pi * self.d ** 3)

    def getTensileStrength(self):
        """
        Calcula e retorna a resistência à tração do material.
        S_ut = A/d^3
        """
        return self.material.A / (self.d ** self.material.m)

    def getShearYieldStrength(self):
        """
        Calcula e retorna o limite de escoamento ao cisalhamento.
        S_sy = x * S_ut
        """
        TensileStrenght = self.getTensileStrength()
        return self.material.x * TensileStrenght 

    def getSafetyFactor(self):
        """
        Calcula e retorna o fator de segurança.
        :param ShearYS: Limite de escoamento ao cisalhamento.
        n_s = S_sy / T_s
        """
        ShearYieldStrength = self.getShearYieldStrength()
        shearStress = self.getShearStress(self.force)
        self.n_s = ShearYieldStrength / shearStress
        return self.n_s
    
    def suggestC(self):
        """
        Calcula o índice da mola.
        :param ShearYS: Limite de escoamento ao cisalhamento.
        n_s = S_sy / T_s
        """
        alpha = self.getShearYieldStrength() / self.lockedSafetyFactor
        beta = 8*(1+self.xi) * self.force / (np.pi * self.d**2)
        C = (2* alpha - beta) / (4*beta) + np.sqrt(((2* alpha - beta) / (4*beta))**2 - 3 *alpha / (4*beta))
        print(f"Índice de mola sugerido: {C}")

################################################ Funções da interface
def on_dropdown_materials_select(event):
    global mola
    selected_material_str = dropdown_material_var.get()
    mola.material = getMaterialById(selected_material_str, available_materials)

    new_diameters = mola.material.diametros_fio
    dropdown_diameters_menu['values'] = new_diameters
    dropdown_diameter_var.set('')

def on_dropdown_diameter_select(event):
    global mola
    mola.d = float(dropdown_diameter_var.get())
    mola.update_spring()

def on_slider_change(value):
    global mola
    mola.C = slider_var.get()
    print(f"Índice da mola (C): {mola.C}")
    mola.update_spring()

def on_number_entry():
    global mola
    try:
        mola.force = float(number_entry.get())
        print(f"Força: {mola.force} N")
        mola.update_spring()
        mola.perform_calculations()
    except ValueError:
        print("Invalid number input")

######################################################## EXECUÇÃO
if __name__ == "__main__":
    mola = Mola()

    root = tk.Tk()
    root.title('Projeto Slinky') 
    root.geometry('400x300+100+0')

    # Dropdown menu
    # Materials
    selected_material = None
    selected_material_str = ""
    dropdown_materials = [mat.id for mat in available_materials]
    dropdown_material_var = tk.StringVar(value=selected_material_str)
    dropdown_materials_menu = ttk.Combobox(root, textvariable=dropdown_material_var, values=dropdown_materials)
    dropdown_materials_menu.bind("<<ComboboxSelected>>", on_dropdown_materials_select)
    dropdown_materials_menu.pack()

    # Diameters
    dropdown_diameter_var = tk.StringVar(value="")
    dropdown_diameters_menu = ttk.Combobox(root, textvariable=dropdown_diameter_var)
    dropdown_diameters_menu.bind("<<ComboboxSelected>>", on_dropdown_diameter_select)
    dropdown_diameters_menu.pack()

    # Slider
    slider_var = tk.DoubleVar(value=4)
    slider = ttk.Scale(root, from_=4, to=12, orient="horizontal", variable=slider_var, command=on_slider_change)
    slider.pack()

    # Number entry
    number_entry = tk.Entry(root)
    number_entry.pack()
    number_button = tk.Button(root, text="Apply Force", command=on_number_entry)
    number_button.pack()

    root.mainloop()


Força: 89.0 N
